#실행전 준비

데이터 출처: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=580

github ipynb 출력 오류로 셀 출력 없는 버전이 업로드 되었습니다.

셀 출력 있는 버전은 colab에 업로드 되어있습니다.

링크: https://colab.research.google.com/drive/1yGjcVJV9NkCCEe5-tHzrC8u4nKhFqSqi?usp=sharing

In [ ]:
# pip installs
!pip install -q -U transformers bitsandbytes protobuf
!pip install -q -U peft trl matplotlib wandb


In [ ]:
#구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#패키지 사용을 위한 파일 업로드
from google.colab import files

src = list(files.upload().values())[0]
open('make_data2.py','wb').write(src)

#라이브러리 import

In [ ]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from datasets import load_dataset,Dataset, DatasetDict

from make_data2 import Datasetup

import numpy as np


In [ ]:
from pathlib import Path
root = "구글 드라이브 경로"

In [ ]:
os.listdir(root)

#json 파일 로드

In [ ]:
from pathlib import Path
import os
import json


ROOT = Path("구글 드라이브 경로")

def iter_json_paths(root: str | Path, pattern: str = "*.json"):
    root = Path(root)
    # rglob 결과를 정렬해서 순서를 고정
    for p in sorted(root.rglob(pattern)):
        yield p

def load_json_file(path: Path, encoding: str = "utf-8"):
    try:
        with path.open("r", encoding=encoding) as f:
            return json.load(f), path
    except json.JSONDecodeError as e:
        return {"__error__": "JSONDecodeError", "message": str(e)}, path
    except OSError as e:
        return {"__error__": "OSError", "message": str(e)}, path

def iter_json_objects(root: str | Path, pattern: str = "*.json", encoding: str = "utf-8"):
    for p in iter_json_paths(root, pattern):
        data, path = load_json_file(p, encoding=encoding)
        yield data, path


In [ ]:
#민사, 형사, 행정 나눠서 json 불러오기

civil_json_list = []
criminal_json_list = []
administration_json_list = []


folder_to_list = {
    "01.민사": civil_json_list,
    "02.형사": criminal_json_list,
    "03.행정": administration_json_list,
}

law_json_list = [civil_json_list, criminal_json_list, administration_json_list]

# 폴더 순회하면서 JSON 로드
# 1차 폴더(01.민사 / 02.형사 / 03.행정)도 sorted로 순서 고정
for folder_entry in sorted(ROOT.iterdir(), key=lambda p: p.name):
    if not folder_entry.is_dir():
        continue

    folder_name = folder_entry.name  # 예: "01.민사"

    if folder_name.startswith("01."):
        target_list = civil_json_list       # 민사
    elif folder_name.startswith("02."):
        target_list = criminal_json_list    # 형사
    elif folder_name.startswith("03."):
        target_list = administration_json_list  # 행정
    else:
        print("매핑되지 않은 폴더:", folder_name)
        continue

    # 연도 폴더들도 정렬
    for year_entry in sorted(folder_entry.iterdir(), key=lambda p: p.name):
        if not year_entry.is_dir():
            continue

        # year_entry 예: /content/.../01.민사/2017
        for obj, p in iter_json_objects(year_entry):
            print(p)
            if obj:
                target_list.append(obj)

In [ ]:
# train, test, val 데이터 나누기

train_law_json_list = []
val_law_json_list = []
test_law_json_list = []

for law_list in law_json_list:#민사 형사 행정 데이터들을 고르게 분배해서 train test val에 나눠 넣기
    n = len(law_list)
    n_train = int(n * 0.8)
    n_val = int(n * 0.93)

    train_law_json_list.extend(law_list[:n_train])
    val_law_json_list.extend(law_list[n_train:n_val])
    test_law_json_list.extend(law_list[n_val:])

print(len(train_law_json_list))
print(len(val_law_json_list))
print(len(test_law_json_list))

In [ ]:
len(train_law_json_list)

In [ ]:
train_law_json_list[0]['info']['caseField']

In [ ]:
test_law_json_list[1]

#데이터 전처리 실험

In [ ]:
import re
stuff = train_law_json_list[1]['assrs']['acusrAssrs']

stuff = str(stuff)
stuff = stuff.replace("\u3000", " ")
stuff = re.sub(r'[:\[\]"{}【】\s]+', ' ', stuff).strip()
stuff = stuff.replace(" ,", ",").replace(",,,",",").replace(",,",",").replace("'","")
stuff = re.sub(r"[ \t]+", " ", stuff)
stuff = re.sub(r"\s*\n\s*", "\n", stuff)
stuff = stuff.strip()
stuff

#Datasetup 클래스 생성 실험

In [ ]:
#업로드한 make_data2 패키지의 Datasetup 클래스가 잘 동작하는지 확인
output1 = Datasetup(train_law_json_list[0])

In [ ]:
output1.prompt

In [ ]:
output1.test_prompt

In [ ]:
output1.token_count

#데이터 분할 생성

밑 코드들은 준비된 데이터를 Datasetup 클래스를 이용하여 프롬프트를 생성하는 코드이다.

다만 colab의 런타임이 자주 끊겨서
데이터들을 나눠서 프롬프트 생성을 진행 후 pkl 파일로 저장했다.



In [ ]:
from tqdm import tqdm
val_data = []
for data in tqdm(val_law_json_list[250:500]):
    val_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/val[250:500].pkl', 'wb') as file:
    pickle.dump(val_data, file)

In [ ]:
from tqdm import tqdm
val_data = []
for data in tqdm(val_law_json_list[500:750]):
    val_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/val[500:750].pkl', 'wb') as file:
    pickle.dump(val_data, file)

In [ ]:
from tqdm import tqdm
val_data = []
for data in tqdm(val_law_json_list[750:1040]):
    val_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/val[750:1040].pkl', 'wb') as file:
    pickle.dump(val_data, file)

In [ ]:
from tqdm import tqdm
test_data = []
for data in tqdm(test_law_json_list[:250]):
    test_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/test[:250].pkl', 'wb') as file:
    pickle.dump(test_data, file)

In [ ]:
from tqdm import tqdm
test_data = []
for data in tqdm(test_law_json_list[250:]):
    test_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/test[250:].pkl', 'wb') as file:
    pickle.dump(test_data, file)

In [ ]:
from tqdm import tqdm
train_data = []
test_data = []
for data in tqdm(train_law_json_list[5500:5750]):
    train_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/train[5500:5750].pkl', 'wb') as file:
    pickle.dump(train_data, file)

In [ ]:
from tqdm import tqdm
train_data = []
test_data = []
for data in tqdm(train_law_json_list[5750:6000]):
    train_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/train[5750:6000].pkl', 'wb') as file:
    pickle.dump(train_data, file)

In [ ]:
from tqdm import tqdm
train_data = []
test_data = []
for data in tqdm(train_law_json_list[6000:6250]):
    train_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/train[6000:6250].pkl', 'wb') as file:
    pickle.dump(train_data, file)

In [ ]:
#train[:1000] 까지완료

from tqdm import tqdm
train_data = []
test_data = []
for data in tqdm(train_law_json_list[6250:6500]):
    train_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/train[6250:6500].pkl', 'wb') as file:
    pickle.dump(train_data, file)

In [ ]:
from tqdm import tqdm
train_data = []
test_data = []
for data in tqdm(train_law_json_list[6500:6750]):
    train_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/train[6500:6750].pkl', 'wb') as file:
    pickle.dump(train_data, file)

In [ ]:
from tqdm import tqdm
train_data = []
test_data = []
for data in tqdm(train_law_json_list[6750:7000]):
    train_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/train[6750:7000].pkl', 'wb') as file:
    pickle.dump(train_data, file)

In [ ]:
from tqdm import tqdm
train_data = []
test_data = []
for data in tqdm(train_law_json_list[4500:4750]):
    train_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/train[4500:4750].pkl', 'wb') as file:
    pickle.dump(train_data, file)

In [ ]:
from tqdm import tqdm
train_data = []
test_data = []
for data in tqdm(train_law_json_list[4750:5000]):
    train_data.append(Datasetup(data))

In [ ]:
import pickle
with open('/content/drive/MyDrive/law_ai/pickle/train[4750:5000].pkl', 'wb') as file:
    pickle.dump(train_data, file)

#파일이 잘 저장되었는지 확인

In [ ]:
with open("/content/drive/MyDrive/law_ai/pickle/train[6000:6250].pkl","rb") as fr:
    data = pickle.load(fr)




In [ ]:
data[1].prompt

#프롬프트의 토큰 분포 확인

In [ ]:
max_1 = 0
count = 0
for data in train_data:
    if data.token_count > max_1:
        max_1 = data.token_count
        if max_1> 8000:
            count += 1

In [ ]:
print(max_1, count)

In [ ]:
# Plot the distribution of token counts again
import matplotlib.pyplot as plt

tokens = [item.token_count for item in train_data]
plt.figure(figsize=(15, 6))
plt.title(f"Token counts: Avg {sum(tokens)/len(tokens):,.1f} and highest {max(tokens):,}\n")
plt.xlabel('Length (tokens)')
plt.ylabel('Count')
plt.hist(tokens, rwidth=0.7, color="skyblue")
plt.show()